In [3]:
import numpy as np
class Mesh:
    def __init__(self):
        self.nodes_dict = {}   # Nodes dict <---> ID: [x y z]
        self.elems_dict = {}   # Elems dict <---> ID: [ne_1, ne_2, ...]
        self.N_nodes = 0
        self.N_elems = 0
        
    def read_mesh(self, mesh_name):
        # Reading nodes
        nodes_file = open(mesh_name + '_nodes.txt', 'r')
        
        for line in nodes_file:
            data_tokens = list(map(float, line.split(' ')))
            node_ID = int(data_tokens[0])
            coord_array = data_tokens[1:]
            self.nodes_dict[node_ID] = coord_array
            
        self.N_nodes = len(self.nodes_dict.keys())
        nodes_file.close()
        
        # Reading elems
        elems_file = open(mesh_name + '_elems.txt', 'r')
        
        for line in elems_file:
            if (line == ''):
                break
            data_tokens = list(map(int, line.replace(' \n', '').split(' ')))
            elem_ID = data_tokens[0]
            elem_nodes = data_tokens[1:]
            self.elems_dict[elem_ID] = elem_nodes
            
        self.N_elems = len(self.elems_dict.keys())
        elems_file.close()
        
        print(f'Mesh from [{mesh_name}] loaded.')
        print('-'*40)
        print(f'Nodes: \t\t{self.N_nodes}')
        print(f'Elements: \t{self.N_elems}')
        
        self.mesh_borders()
    
    def get_mesh(self):
        return self.nodes_dict, self.elems_dict
    
    def mesh_borders(self):
        self.min_x = 1e9
        self.max_x = -1e9
        self.min_y = 1e9
        self.max_y = -1e9
        
        for node_id in list(self.nodes_dict.keys()):
            node_x, node_y, node_z = tuple(self.nodes_dict[node_id])
            if (node_x > self.max_x):
                self.max_x = node_x
            if (node_x < self.min_x):
                self.min_x = node_x
            if (node_y > self.max_y):
                self.max_y = node_y
            if (node_y < self.min_y):
                self.min_y = node_y
                
    def left_bottom(self):
        for node_id in list(self.nodes_dict.keys()):
            node_x, node_y, node_z = tuple(self.nodes_dict[node_id])
            if (node_x == self.min_x and node_y == self.min_y):
                return node_id
                
    def left_upper(self):
        for node_id in list(self.nodes_dict.keys()):
            node_x, node_y, node_z = tuple(self.nodes_dict[node_id])
            if (node_x == self.min_x and node_y == self.max_y):
                return node_id
                
    def right_bottom(self):
        for node_id in list(self.nodes_dict.keys()):
            node_x, node_y, node_z = tuple(self.nodes_dict[node_id])
            if (node_x == self.max_x and node_y == self.min_y):
                return node_id
                
    def right_upper(self):
        for node_id in list(self.nodes_dict.keys()):
            node_x, node_y, node_z = tuple(self.nodes_dict[node_id])
            if (node_x == self.max_x and node_y == self.max_y):
                return node_id

    def elem_center(self, elem_id):
        elem_nodes = self.elems_dict[elem_id]
        
        x1 = self.nodes_dict[elem_nodes[0]][0]
        y1 = self.nodes_dict[elem_nodes[0]][1]
        
        x2 = self.nodes_dict[elem_nodes[1]][0]
        y2 = self.nodes_dict[elem_nodes[1]][1]
        
        x3 = self.nodes_dict[elem_nodes[2]][0]
        y3 = self.nodes_dict[elem_nodes[2]][1]

        x_center = (x1 + x2 + x3)/3
        y_center = (y1 + y2 + y3)/3

        return (x_center, y_center)

    def calc_filter(self, filter_R):
        H_lines = []
        
        for elem_id in list(self.elems_dict.keys()):
            x_center, y_center = self.elem_center(elem_id)
            neighbors_dist = np.zeros(self.N_elems)
            for i, elem_id_ in enumerate(list(self.elems_dict.keys())):
                x_cent, y_cent = self.elem_center(elem_id_)
                distance = np.sqrt((x_center - x_cent)**2 + (y_center - y_cent)**2)
                neighbors_dist[i] = max(0, filter_R - distance)
            H_lines.append(neighbors_dist)

        H = np.stack(H_lines, axis = 0)
        sH = np.sum(H, axis = 1)

        return H, sH